In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from lmfit import Parameters, minimize, report_fit

In [3]:
def lorentzian( w, w0, w_p, gam ):
    return w * gam * pow(w_p,2) / ( (w*gam)**2 + ( pow(w,2) - pow(w0,2) )**2)

def lorentz_dataset(params, i, w):
    """Calculate Lorentzian lineshape from parameters for data set."""
    w0 = params['center_%i' % (i+1)]
    w_p = params['plasma_frequency_%i' % (i+1)]
    gam = params['gamma_%i' % (i+1)]
    return lorentzian(w, w0, w_p, gam)

def objective(params, w, data):
    """Calculate total residual for fits of Gaussians to several data sets."""
    ndata, _ = data.shape
    resid = 0.0*data[:]

    # make residual per data set
    for i in range(ndata):
        resid[i, :] = data[i, :] - lorentz_dataset(params, i, w)

    # now flatten this to a 1D array, as minimize() needs
    return resid.flatten()

In [46]:
dataset = np.loadtxt('E2_Silicon.dat', unpack=True )

In [47]:
dataset.shape

(2, 6801)

In [39]:
for x, y in enumerate(data):
    print(x,y)

0 [ 6000.  6005.  6010. ... 39990. 39995. 40000.]
1 [ 0.          0.          0.         ... 12.49904916 12.49419041
 12.48933223]


In [34]:
data[:,:]

array([[6.00000000e+03, 6.00500000e+03, 6.01000000e+03, ...,
        3.99900000e+04, 3.99950000e+04, 4.00000000e+04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.24990492e+01, 1.24941904e+01, 1.24893322e+01]])

In [29]:
for i in np.arange(20):
    params = Parameters()

In [31]:
max_center=np.argmax(data[:,1])
maxY=data[max_center]
maxX=w[max_center]

In [32]:
max_center

0

In [21]:
fit_params = Parameters()
for iy, y in enumerate(data):
    fit_params.add('center_%i' % (iy+1), value=max_center, min=0.0, max=200)
    fit_params.add('plasma_frequency_%i' % (iy+1), value=np.sqrt(maxX*maxY), min=0, max=10000)
    fit_params.add('gamma_%i' % (iy+1), value=1, min=0, max=10000)

In [22]:
out = minimize(objective, fit_params, args=(w, data))
report_fit(out.params)

ValueError: not enough values to unpack (expected 2, got 1)

In [24]:
import matplotlib.pyplot as plt
import numpy as np

from lmfit import Parameters, minimize, report_fit


def gauss(x, amp, cen, sigma):
    """Gaussian lineshape."""
    return amp * np.exp(-(x-cen)**2 / (2.*sigma**2))


def gauss_dataset(params, i, x):
    """Calculate Gaussian lineshape from parameters for data set."""
    amp = params['amp_%i' % (i+1)]
    cen = params['cen_%i' % (i+1)]
    sig = params['sig_%i' % (i+1)]
    return gauss(x, amp, cen, sig)


def objective(params, x, data):
    """Calculate total residual for fits of Gaussians to several data sets."""
    ndata, _ = data.shape
    resid = 0.0*data[:]

    # make residual per data set
    for i in range(ndata):
        resid[i, :] = data[i, :] - gauss_dataset(params, i, x)

    # now flatten this to a 1D array, as minimize() needs
    return resid.flatten()

In [56]:
x = np.linspace(-1, 2, 151)
data = []
for i in np.arange(5):
    params = Parameters()
    amp = 0.60 + 9.50*np.random.rand()
    cen = -0.20 + 1.20*np.random.rand()
    sig = 0.25 + 0.03*np.random.rand()
    dat = gauss(x, amp, cen, sig) + np.random.normal(size=x.size, scale=0.1)
    print(gauss(x, amp, cen, sig))
    data.append(dat)
data = np.array(data)

[5.90294047e-04 8.24953301e-04 1.14561318e-03 1.58086330e-03
 2.16769565e-03 2.95358839e-03 3.99898076e-03 5.38017414e-03
 7.19268478e-03 9.55505955e-03 1.26131467e-02 1.65447865e-02
 2.15648563e-02 2.79305600e-02 3.59468132e-02 4.59715157e-02
 5.84204539e-02 7.37715158e-02 9.25678471e-02 1.15419529e-01
 1.43003315e-01 1.76059956e-01 2.15388618e-01 2.61837948e-01
 3.16293379e-01 3.79660391e-01 4.52843519e-01 5.36721137e-01
 6.32116195e-01 7.39763350e-01 8.60273176e-01 9.94094402e-01
 1.14147539e+00 1.30242628e+00 1.47668351e+00 1.66367839e+00
 1.86251168e+00 2.07193586e+00 2.29034701e+00 2.51578744e+00
 2.74596044e+00 2.97825778e+00 3.20979997e+00 3.43748910e+00
 3.65807299e+00 3.86821923e+00 4.06459678e+00 4.24396247e+00
 4.40324932e+00 4.53965338e+00 4.65071566e+00 4.73439575e+00
 4.78913417e+00 4.81390070e+00 4.80822659e+00 4.77221939e+00
 4.70655948e+00 4.61247876e+00 4.49172213e+00 4.34649377e+00
 4.17939018e+00 3.99332309e+00 3.79143537e+00 3.57701319e+00
 3.35339801e+00 3.123901

In [52]:
data[0,0]

0.05242025658336122

In [45]:
fit_params = Parameters()
for iy, y in enumerate(data[0]):
    fit_params.add('amp_%i' % (iy+1), value=0.5, min=0.0, max=200)
    fit_params.add('cen_%i' % (iy+1), value=0.4, min=-2.0, max=2.0)
    fit_params.add('sig_%i' % (iy+1), value=0.3, min=0.01, max=3.0)

0 [ 5.24202566e-02 -1.10840385e-01 -2.01899399e-01 -7.30962401e-02
 -1.31903729e-01 -1.21692115e-01  1.52009010e-02 -1.06401159e-02
 -1.43693186e-01  6.46868338e-02  4.64985033e-03 -6.31516666e-03
 -1.99980113e-02 -9.61424459e-02  7.23573830e-03 -3.67947563e-02
  1.38084437e-01 -2.00009190e-01 -2.63848581e-02 -7.70683121e-03
 -1.31429054e-01  1.55293794e-02 -4.33845034e-03 -7.50949971e-02
 -5.78480860e-02  8.41439129e-02  7.18519600e-02  1.79976050e-01
 -5.13536889e-02  1.00582989e-01  1.66049870e-01 -6.95098489e-02
  1.06057529e-01  6.49059276e-02  8.68956925e-02  6.91366390e-02
 -6.56646335e-02  1.10695136e-01 -2.58844823e-02 -1.21175801e-02
  1.68918523e-01  2.14684852e-01  3.18805119e-01  3.82312437e-01
  4.47070844e-01  3.49722566e-01  5.21659827e-01  5.81864556e-01
  8.44546874e-01  9.83621195e-01  1.05495692e+00  1.27861061e+00
  1.38912160e+00  1.57662608e+00  2.05280649e+00  2.34618874e+00
  2.60675779e+00  2.80922483e+00  3.15796391e+00  3.74090734e+00
  3.92492819e+00  4.347